In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


Opens and reads the file.

In [2]:
file = r.TFile("generatedComponents.root","READ")

Warning in <TFile::Init>: file generatedComponents.root probably not closed, trying to recover
Info in <TFile::Recover>: generatedComponents.root, recovered key TTree:tree at address 64540
Warning in <TFile::Init>: successfully recovered 1 keys


In [3]:
tree = file.Get("tree") # Getting the tree 

In [4]:
# Creating TLorentzVector which we do not actually need in stationary W case
eTLV = r.TLorentzVector()
nTLV = r.TLorentzVector()

In [5]:
# Initializing 4-element array to store 4-vectors
wArr = np.zeros(4)
eArr = np.zeros(4)
nArr = np.zeros(4)

In [6]:
# Taking already filled branches from the file
# Now we need to add another branch for neutrino
tree.SetBranchAddress("wBoson", wArr)
tree.SetBranchAddress("electron", eArr)

4

In [7]:
# Initializing branch for neutrino components
tree.Branch("neutrino", nArr, "neArray[4]/D")

# Minimisation with TMinuit

Now we need to use TMinuit to calculate neutrino Pz.

We need to define functions, used to work with TMinuit, in C++ cell. This can be done by using the %%cpp cell magic. This transforms a cell into a C++ cell. To define functions, we need to start the cell with `%%cpp -d`. If we want to call the functions, we can do it either in another C++ cell, but we would need to start the cell with only `%%cpp`.

However we can call these functions from python cell, too. See cells below.

See variable `initVal` in the function `minimization_func`. This is done to get values with the right sign for values of electron Pz which are lesser than 1 but larger than 0.


In [8]:
%%cpp -d   

void fcn(int &npar, double *gin, double &f, double *par, int iflag){
    double x = par[0];
    double y = par[1];
    double nPz = par[2];

    double wMass = TMath::Sqrt(x) + TMath::Sqrt(y + pow(nPz,2));

    f = pow(wMass-80.3,2);
};

double minimization_func(double *wArr, double *elArr){
    double ePx = elArr[0];
    double ePy = elArr[1];
    double ePz = elArr[2];
    
    double initVal;
    
    if(abs(ePz) < 1 && ePz > 0){
        initVal = -0.0001;
    } else {
        initVal = -ePz + 1;
    }
    
    double pTSqEl = pow(ePx,2) + pow(ePy,2);
    double pTotSqEl = pTSqEl + pow(ePz,2);  
    
    TMinuit *gMinuit = new TMinuit(3);
    gMinuit->SetFCN(fcn);
    gMinuit->DefineParameter(0, "pTotSqEl", pTotSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(1, "pTSqEl", pTSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(2, "nPz", initVal, 0.005, 0., 0.);   
    gMinuit->FixParameter(0);
    gMinuit->FixParameter(1);    
    gMinuit->Command("MIGRAD");

    double nPz, nPzErr;
    gMinuit->GetParameter(2, nPz, nPzErr);
    return nPz;

};

In [9]:
n = tree.GetEntries() # Number of entries in tree (number of different events)

for i in range(n):
    tree.GetEntry(i)
    ePx = tree.GetListOfLeaves().At(1).GetValue(0)
    ePy = tree.GetListOfLeaves().At(1).GetValue(1)
    
    nArr[0] = -ePx
    nArr[1] = -ePy
    nArr[2] = r.minimization_func(wArr, eArr)
    nArr[3] = r.TMath.Sqrt(nArr[0]**2 + nArr[1]**2 + nArr[2]**2)
    
    tree.GetBranch("neutrino").Fill() # Filling just "neutrino" branch

 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 pTotSqEl     1.61202e+03  1.00000e+00     no limits
     2 pTSqEl       1.61141e+03  1.00000e+00     no limits
     3 nPz         -1.00000e-04  5.00000e-03     no limits
 **********
 **    1 **FIX           1
 **********
 **********
 **    2 **FIX           2
 **********
 **********
 **    3 **MIGRAD 
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-04
 FCN=2.06903e-05 FROM MIGRAD    STATUS=INITIATE       15 CALLS          16 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS       STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  pTotSqEl     1.61202e+03     fixed    
   2  pTSqEl       1.61141e+03     fixed    
   3  nPz         -5.00100e-01   5.00000e-03  -5.00000e-01   1.

Error in <TBranch::TBranch::WriteBasketImpl>: basket's WriteBuffer failed.

Error in <TBranch::TBranch::Fill>: Failed to write out basket.



In [10]:
# Scanning to see if everything works alright
tree.Scan()

4000

***********************************************************
*    Row   * Instance * wBoson.wB * electron. * neutrino. *
***********************************************************
*        0 *        0 *         0 * 36.022545 * -36.02254 *
*        0 *        1 *         0 * 17.713931 * -17.71393 *
*        0 *        2 *         0 * 0.7844337 * -0.782482 *
*        0 *        3 *      80.3 *     40.15 * 40.149961 *
*        1 *        0 *         0 * -0.710591 * 0.7105916 *
*        1 *        1 *         0 * 4.0137899 * -4.013789 *
*        1 *        2 *         0 * -39.94254 * 39.942909 *
*        1 *        3 *      80.3 *     40.15 * 40.150360 *
*        2 *        0 *         0 * 6.5922598 * -6.592259 *
*        2 *        1 *         0 * -23.09224 * 23.092244 *
*        2 *        2 *         0 * 32.176277 * -32.17596 *
*        2 *        3 *      80.3 *     40.15 * 40.149748 *
*        3 *        0 *         0 * -21.85706 * 21.857069 *
*        3 *        1 *         0 * 1.41

Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or q to quit ==> Type <CR> to continue or 